In [1]:
import ginza
import spacy
nlp = spacy.load('ja_ginza')

In [2]:
def is_ADJ(token):
    # pos = token.tag_.split("-")
    # try:
    #     if pos[0] == "動詞" or pos[0] == "形容詞" or pos[2] == "サ変可能":
    #         # print(token.lemma_)
    #         return True
    #     else:
    #         False
    # except:
    #     return False
    if token.pos_ == "ADJ":
        return True
    return False

In [8]:
text = "死者は良いですね"

In [15]:
def dependent_ADJ_NOUN(text):
    doc = nlp(text)
    for token in doc:
        if is_ADJ(token):
            # print(token)
            for child in token.children:
                if "NOUN" == child.pos_:
                    print(child, token)


In [16]:
dependent_ADJ_NOUN(text)

死者 良い


In [14]:
from spacy import displacy
displacy.render(nlp(text), style='dep', jupyter=True, options={'compact':True, 'distance': 90})

In [17]:
class Utterance:
    utt_level = ["Wrong information", "Semantic error", "Uninterpretable", "Grammatical error"]
    def __init__(self, did, sp, utt, errors, type_) -> None:
        self.did = did
        self.sp = sp
        self.utt = utt
        self.errors = errors
        self.type_ = type_

        self.is_utt_error = False
        for e in Utterance.utt_level:
            if self.is_error_included(e):
                self.is_utt_error = True
    
    def __str__(self):
        return "{0}: {1}".format(self.sp, self.utt)

    def is_system(self):
        return True if self.sp=="S" else False
    
    def is_error_included(self, error):
        # Null 対応
        if not self.errors:
            return False
        if isinstance(error, list):
            is_inclued = False
            for e in error:
                is_inclued = is_inclued or (e in self.errors)
            return is_inclued
        elif isinstance(error, str):
            return error in self.errors
        else:
            return False

    def is_exist_error(self):
        return True if self.errors else False
    
    def is_type_included(self, type_):
        return type_ in self.type_
    
    def is_exist_type(self):
        return True if self.type_ else False
    
    def is_utt_level_error(self):
        return self.is_utt_error
    


def read_conv(path:str, datalist:list):
    convs = []
    for p in datalist:
        datapath = Path(path + p + '/')
        for file_ in datapath.glob("*.json"):
            conv = []
            with open(file_, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    sp = t["speaker"]
                    utt = t["utterance"]
                    errors = t["error_category"]
                    type_ = t["type"]
                    one = Utterance(did, sp, utt, errors, type_)
                    conv.append(one)
            convs.append(conv)
    return convs  

In [23]:
from pathlib import Path
import json
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [24]:
error = "Semantic error"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [26]:
for utt in sys_utt:
   dependent_ADJ_NOUN(utt) 

海 大丈夫
海 危険
野菜 低い
カロリー 低い
カロリー 欲しい
ラーメン 美味しい
麺 美味しい
パスタ 好き
パスタ 好き
系 好き
海 楽しい
海 サイコー
海 素晴らしい
海 素晴らしい
台風 凄い
熱中症 いい
熱中症 大丈夫
うがい 大切
予防 大切
予防 難しい
プレゼント 嬉しい
海外旅行 楽しい
一緒 いい
味噌汁 いい
ご飯 いい
納豆 最高
食べ物 美味しい
歯ごたえ いい
メロン 好き
スイカ 大好き
メロン 大好き
メロン 好き
糖度 高い
曲 好き
酵素 強い
猛暑 欲しい
猛暑 嬉しい
猛暑 欲しい
冷夏 多い
冷夏 高い
野菜 高い
冷夏 多い
冷夏 高い
野菜 高い
野菜 多い
質 多い
元気 良い
雨 大丈夫
雨 眠い
もったい ない
一人 いい
雨 悪い
足元 悪い
調子 良い
腰 丈夫
腰 優しい
腰 痛い
筋肉 痛い
ま ない
しっぽ 短い
糖度 高い
果物 高い
カロリー 高い
カロリー よい
熱中症 いい
死者 良い
気持ち いい
天気 いい
気持ち いい
天気 良い
ホント 良い
天気 いい
晴れ いい
晴れ いい
気持ち いい
ソフトドリンク 多い
量 多い
無料 安い
ソフトドリンク 安い
笑い 楽しい
プレゼント 嬉しい
チャンプル 美味しい
チャンプル 美味しい
こと 多い
わけ 好き
熱中症 いい
予防 大切
胃 綺麗
酒 有名
頭痛 凄い
クラス いい
仲 いい
仲 いい
登山 好き
お 好き
とき 必要
準備 必要
山 好き
ところ ない
ごまかし いい
ごまかし うまい
ゲーム 豊富
夜 いい
人 少ない
画面 いい
バレー 有名
バレー 熱い
砂浜 熱い
やけど 深い
海 穏やか
海 楽しい
湖 綺麗
透明度 欲しい
透明度 良い
公式 いい
読み いい
喉 痛い
とき いい
首 大事
首 温かい
首 欲しい
日本人 好き
本当 好き
ラーメン 好き
マンゴー 濃い
味 濃い
果物 いい
生 いい
冷凍 美味しい
果物 良い
果物 良い
音楽 好き
お 好き
相性 悪い
スイカ 大好き
酸味 おいしい
犬 いい
芸 薄い
芸 小さい
芸 可愛い
海 好き
予防 肝心
インターネット 多い
量 多い
海 穏やか
海 温かい
海 熱い
海 好き
機嫌 おかしい
機嫌 悪く
機嫌 悪い
機嫌 いい
初耳 